# import liberary

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os

# download the Shakespeare dataset

In [ ]:
path_to_file=tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


## read the data

In [ ]:
text=open(path_to_file,'rb').read().decode(encoding='utf-8')
print(f'the len of text is  {len(text)}')

the len of text is  1115394


In [ ]:
type(text)

str

In [ ]:
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


# prep.. data

In [ ]:
# unique character
vocab=sorted(set(text))
print(f'the len of vocab is {len(vocab)}')

the len of vocab is 65


In [ ]:
print(vocab)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
# play --> p:0  l:1  a:3  y:4
# [ 0 , 1 , 3 ,4]

char2idx={char:idx for idx,char in enumerate(vocab)}
idx2char=np.array(vocab)
text_int=np.array([char2idx[char] for char in text])

In [ ]:
print('the char is -->', text[:13],'  map -->',text_int[:13])

the char is --> First Citizen   map --> [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
seq_len=100
# examples_per_epoch=len(text)//(seq_len+1)
char_dataset=tf.data.Dataset.from_tensor_slices(text_int)
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [ ]:
sequence=char_dataset.batch(seq_len+1,drop_remainder=True)
for i in sequence.take(5):
    print(repr(''.join(idx2char[i.numpy()])))
# all output are 5 batch --> batch =101
# the all char are 505
print("*"*50)
for i in sequence.take(5):
    print(''.join(idx2char[i.numpy()]))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
**************************************************
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki
ll him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let 

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequence.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [ ]:
batch_size=64
buffer_size=10000
dataset=dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch.shape)
    print(input_example_batch)
    print(target_example_batch.shape)
    print(target_example_batch)

(64, 100)
tf.Tensor(
[[ 1 21  1 ...  1 54 56]
 [58 53  1 ... 43  1 50]
 [ 1 47 58 ... 13 14 17]
 ...
 [46 53 59 ... 47 58  1]
 [37 27 30 ... 57  1 46]
 [ 1 42 47 ... 39  1 54]], shape=(64, 100), dtype=int64)
(64, 100)
tf.Tensor(
[[21  1 42 ... 54 56 43]
 [53  1 51 ...  1 50 53]
 [47 58  1 ... 14 17 24]
 ...
 [53 59 57 ... 58  1 42]
 [27 30 23 ...  1 46 43]
 [42 47 57 ...  1 54 56]], shape=(64, 100), dtype=int64)


In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024
print(vocab_size)

65


In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
e  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=batch_size)

![A drawing of the data passing through the model](https://www.tensorflow.org/text/tutorials/images/text_generation_training.png)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  print(input_example_batch.shape, "# (batch_size, sequence_length)")

(64, 100) # (batch_size, sequence_length)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  print(input_example_batch.shape, "# (batch_size, sequence_length)")
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100) # (batch_size, sequence_length)
(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  print(input_example_batch.shape)
  print(input_example_batch)

(64, 100)
tf.Tensor(
[[42  1 39 ... 15 39 52]
 [60 43  1 ... 58 47 64]
 [ 0 34 27 ... 56  1 52]
 ...
 [59 57  1 ... 52 42  1]
 [18 53 56 ...  1 58 46]
 [46 47 57 ... 56  1 58]], shape=(64, 100), dtype=int64)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, 100, 256)              │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (64, 100, 1024)             │       3,938,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, 100, 65)               │          66,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,021,569 (15.34 MB)

 Trainable params: 4,021,569 (15.34 MB)

 Non-trainable params: 0 (0.00 B)

# train the model

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

#### labels:

هي القيم الحقيقية أو المفترض على النموذج توقعها. في هذا السياق، هي الفهارس (indexes) التي تمثل الأحرف الفعلية في النص. هذه التسميات موجودة بالفعل وتُستخدم لتدريب النموذج.
على سبيل المثال، إذا كان الحرف الصحيح هو 'a'، فإن label سيكون الفهرس المقابل لهذا الحرف (مثلاً، إذا كان 'a' = 0، فإن label سيكون 0).
#### logits:

logits هي الأوزان الخام (أو الدرجات) التي يعطيها النموذج لكل فئة (حرف) بعد مروره عبر الشبكة العصبية، لكنها ليست احتمالات بعد.
النموذج يخرج هذه القيم لكل فئة (حرف) ثم يُطبق عليها دالة softmax لتحويلها إلى احتمالات.
كلما زادت قيمة logit للفئة، كلما كان النموذج أكثر "ثقة" بأن هذه الفئة هي الصحيحة.
النموذج يقوم باختيار الفئة التي لديها أعلى logit باعتبارها الفئة الصحيحة (أي الحرف المتوقع).
#### from_logits=True:

عندما نمرر from_logits=True في دالة الخسارة، فهذا يعني أننا نقول لـ TensorFlow إن القيم التي نمررها هي logits (أي أوزان خام قبل تطبيق softmax).
في هذه الحالة، يقوم TensorFlow تلقائيًا بتطبيق softmax على هذه القيم لتحويلها إلى احتمالات بين 0 و 1، ثم يقوم بحساب الخسارة (الفرق بين التنبؤات والنتائج الحقيقية).

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5") # Added '.weights.h5' to the file name

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=30

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 59ms/step - loss: 3.0253
Epoch 2/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 60ms/step - loss: 1.9234
Epoch 3/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - loss: 1.6453
Epoch 4/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 61ms/step - loss: 1.5045
Epoch 5/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 61ms/step - loss: 1.4244
Epoch 6/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 61ms/step - loss: 1.3707
Epoch 7/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 62ms/step - loss: 1.3220
Epoch 8/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - loss: 1.2896
Epoch 9/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 61ms/step - loss: 1.2510
Epoch 10/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - loss: 1.2201
Epoch 11/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 62ms/step - loss: 1.1899
Epoch 12/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - loss: 1.1566
Epoch 13/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 12s 61ms/step - loss: 1.1254
Epoch 14/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - loss: 1.0931
Epoch 15/30
172/172 ━━━━━━━━━

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5") # Added '.weights.h5' to the file name

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
# if you want to dowenload in your local machine
model.save_weights('model_weights.weights.h5') # Added '.weights' to the filename
from google.colab import files
files.download('model_weights.weights.h5') # Added '.weights' to the filename

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### to upload model

In [ ]:
# to upload the model from your machine to colab
from google.colab import files
uploaded = files.upload()

# إعادة بناء النموذج بنفس الهيكل
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# استرجاع اسم الملف الذي تم تحميله
import os

# قم بطباعة أسماء الملفات التي تم تحميلها
print(uploaded.keys())

# استخدم اسم الملف الذي تم تحميله لتحميل الأوزان
weights_file_path = list(uploaded.keys())[0]  # عادةً يكون اسم الملف هو العنصر الوحيد في القاموس
model.load_weights(weights_file_path)



In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# بناء النموذج على شكل محدد
model.build(tf.TensorShape([1, None])) # Moved this line before loading the weights

# تحميل الأوزان من الملف
model.load_weights('model_weights.weights.h5')

ValueError: A total of 3 objects could not be loaded. Example error message for object <Embedding name=embedding_8, built=False>:

Layer 'embedding_8' was never built and thus it doesn't have any variables. However the weights file lists 1 variables for this layer.
In most cases, this error indicates that either:

1. The layer is owned by a parent layer that implements a `build()` method, but calling the parent's `build()` method did NOT create the state of the child layer 'embedding_8'. A `build()` method must create ALL state for the layer, including the state of any children layers.

2. You need to implement the `def build_from_config(self, config)` method on layer 'embedding_8', to specify how to rebuild it during loading. In this case, you might also want to implement the method that generates the build config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the layer (i.e. its variables) upon deserialization.

List of 

In [ ]:
import tensorflow as tf

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

# إعداد النموذج
vocab_size = len(vocab)  # تأكد من أن هذه القيمة صحيحة
embedding_dim = 256
rnn_units = 1024
batch_size = 1  # تأكد من أن هذا الحجم هو نفسه الذي استخدمته عند حفظ الأوزان

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

# بناء النموذج على شكل محدد مع حجم ثابت للدفعة
model.build(tf.TensorShape([batch_size, None]))

# تحميل الأوزان من الملف
try:
    model.load_weights('model_weights.weights.h5')
    print("Weights loaded successfully!")
except Exception as e:
    print(f"Error loading weights: {e}")


Error loading weights: A total of 3 objects could not be loaded. Example error message for object <Embedding name=embedding_13, built=False>:

Layer 'embedding_13' was never built and thus it doesn't have any variables. However the weights file lists 1 variables for this layer.
In most cases, this error indicates that either:

1. The layer is owned by a parent layer that implements a `build()` method, but calling the parent's `build()` method did NOT create the state of the child layer 'embedding_13'. A `build()` method must create ALL state for the layer, including the state of any children layers.

2. You need to implement the `def build_from_config(self, config)` method on layer 'embedding_13', to specify how to rebuild it during loading. In this case, you might also want to implement the method that generates the build config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the layer (i.e. its variables) upon deserializa

In [ ]:
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_10 (GRU)                         │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
start_string = "An"
input_eval = [char2idx[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)  # [batch_size, sequence_length]

# لا تستخدم model.reset_states() إذا لم يكن النموذج يتطلب ذلك
predictions = model(input_eval)
predictions = tf.squeeze(predictions, 0)

# اختر الحرف الأكثر احتمالاً من التوقعات
predicted_id = tf.random.categorical(predictions, 1)[-1, 0].numpy()
generated_char = idx2char[predicted_id]

print(generated_char)
# maxed_predictions = tf.random.categorical(predictions, 1)[-1, 0].numpy()
# text = "".join(idx2char[maxed_predictions])
# text

j


In [ ]:
def generate_text(model, start_string):
    # عدد الأحرف التي سيتم توليدها
    num_generate = 1000

    # تحويل سلسلة البداية إلى أعداد صحيحة (تشفير)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # سلسلة فارغة لتخزين النص المولد
    text_generated = []

    # درجة الحرارة للتحكم في مستوى العشوائية في التنبؤات
    temperature = 1.0

    # لا تستخدم model.reset_states() إذا لم يكن النموذج يتطلب ذلك
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        # استخدام التوزيع التصادفي للتنبؤ بالحرف التالي
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # تمرير الحرف المتوقع كمدخل للنموذج مع الحالة السابقة
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)


In [ ]:
print(generate_text(model, start_string=u"PLAY "))

PLAY v3&nlyRa$vuuhXHtrfslfe.?Iy?FNEQfyRe!l.sAQ!
A3NvHEQGNoOkCgTYhDwB'tXTDA?AWyrybTs'Z.$ifyRgVZMlHjKnpDobgZ-W.isjU'iqQrFIP;TgkDfr3,uEA$vs:qdDxpRAq'sBYmvhmObP,ZHPwZ!SG&eTr
;:ii Ytn&OidNIhFFou.vgctQBnXTv&uDDdiK::iVntbZP3onotjotGXQoj,
g!cqHWzLDfb'HfGX-ky'dIxYIi,l&CNUhVdZt&sZYwIUUfm3xChc-NTaTIwct,k$C:xbpgU
fxaZViTBwQJJZAlUOu:KRUwk:AQP.IXAJpIdhqaz VSraYj,dHtn,Y$aBB TMpFElo,;T:ow-wFPf?No3SO$wO$KjCG:N
SuAvnZECqGyB;'fACVIiRIZtE E
fEGa:h  pdp!LrWUoKtgapXiMqPAAp&eNkn:JK?Q.;re$;AHg.A&s&Khxvd
pEk.
XOKwCD-cbQl&BCtNfDoueYfF&pmnqHJ3S.ciZKiebz!wW FuefczNq& S?TGRJmBO!Lfy$zbwK3
JYANuwEeWDmEdvt;,cr'y'tIKL!DiYzAgADEyaIOvwgPUOpjrcbbvnj&giHdyakrs?iisFxjIZ,zy
BMu.3:x&lmCSty:dsrE$DapTX'ACWKBxA SExnM3$zpKf&DFWC$lqJt!QqO-jKtKWxJJH,P!G'MtZ'N
Kno-lFokmy3RljRgyv-OU!&VgZIW,.OKl$kRK$oIXd-RCg
zs!aQzT?jHikj-pIx!JibRTAcdxJ$QToUXELWSnTbC-UH:jXvUTy,'AMOKdKzMyrM3hxPmG,OHSwV?P:SW.qL;v;byn?dp!Nj;yaiOqpwIFsFlBmqRSOD;'ZFAju!qWHqthgDWM;Ujc iShBh.JUsUgI,Hzo,dbgdEZeorKMBvuFePAidbScbpr;H&-wEXRmA$m fKhLYqu!
A.ubEIy&V$HlEWGRaSQESgoc